# Import necessary packages

In [51]:
import tensorflow as tf
import pprint
pp = pprint.PrettyPrinter()

# Global config

In [61]:
vocab_size=20000
max_sentence_length = 40
word_embedding_vector_dimension = 20

# downloading the dataset

In [14]:
!wget https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json

--2022-09-06 23:57:29--  https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.198.80, 216.58.205.208, 172.217.21.16, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.198.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5,4M) [application/json]
Saving to: 'sarcasm.json.1'

     0K .......... .......... .......... .......... ..........  0%  454K 12s
    50K .......... .......... .......... .......... ..........  1%  849K 9s
   100K .......... .......... .......... .......... ..........  2% 1,44M 7s
   150K .......... .......... .......... .......... ..........  3% 1,54M 6s
   200K .......... .......... .......... .......... ..........  4% 1,63M 6s
   250K .......... .......... .......... .......... ..........  5% 1,58M 5s
   300K .......... .......... .......... .......... ..........  6% 1,60M 5s
   350K .......... .......... .......... .....

# Importing the Database

In [53]:
import json

# Load the JSON file
with open("./sarcasm.json", 'r') as f:
    datastore = json.load(f)
datastore[0]
print("number of rows =", len(datastore))

number of rows = 26709


# headline as data and is_sarcastic as label

In [54]:
import numpy as np
headlines , is_sarcastic = [] ,  []
for row in datastore :
    headlines.append(row['headline'])
    is_sarcastic.append(row['is_sarcastic'])
headlines=np.array(headlines)
is_sarcastic=np.array(is_sarcastic)
print("X = ",headlines[0:5])
print("Y =", is_sarcastic[0:5])

X =  ["former versace store clerk sues over secret 'black code' for minority shoppers"
 "the 'roseanne' revival catches up to our thorny political mood, for better and worse"
 "mom starting to fear son's web series closest thing she will have to grandchild"
 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas'
 'j.k. rowling wishes snape happy birthday in the most magical way']
Y = [0 0 1 1 0]


# Split the data into training and test

In [55]:
from sklearn.model_selection import train_test_split
headlines_train , headlines_test , is_sarcastic_train, is_sarcastic_test = train_test_split(headlines , is_sarcastic , test_size=0.2 , random_state=27 )


# Initialize The Tokenizer 

In [56]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(oov_token="<UNK>" , num_words=vocab_size)
tokenizer.fit_on_texts(headlines_train)
word_index = tokenizer.word_index
word_index

{'<UNK>': 1,
 'to': 2,
 'of': 3,
 'the': 4,
 'in': 5,
 'for': 6,
 'a': 7,
 'on': 8,
 'and': 9,
 'with': 10,
 'is': 11,
 'new': 12,
 'trump': 13,
 'man': 14,
 'from': 15,
 'at': 16,
 'about': 17,
 'you': 18,
 'this': 19,
 'after': 20,
 'by': 21,
 'out': 22,
 'up': 23,
 'how': 24,
 'be': 25,
 'it': 26,
 'as': 27,
 'that': 28,
 'not': 29,
 'your': 30,
 'are': 31,
 'his': 32,
 'what': 33,
 'he': 34,
 'all': 35,
 'who': 36,
 'has': 37,
 'more': 38,
 'just': 39,
 'will': 40,
 'one': 41,
 'into': 42,
 'year': 43,
 'why': 44,
 'have': 45,
 'report': 46,
 'over': 47,
 'area': 48,
 'donald': 49,
 'u': 50,
 'says': 51,
 'first': 52,
 's': 53,
 'day': 54,
 'can': 55,
 'woman': 56,
 'her': 57,
 'time': 58,
 'like': 59,
 'no': 60,
 "trump's": 61,
 'old': 62,
 'get': 63,
 'life': 64,
 'obama': 65,
 'an': 66,
 'off': 67,
 'people': 68,
 'now': 69,
 'house': 70,
 'make': 71,
 'still': 72,
 "'": 73,
 'than': 74,
 'women': 75,
 'white': 76,
 'back': 77,
 'my': 78,
 'was': 79,
 'i': 80,
 'clinton': 81,
 '

# Tokenize the training and the test data

In [57]:
headlines_train_sequence = tokenizer.texts_to_sequences(headlines_train)
headlines_test_sequence = tokenizer.texts_to_sequences(headlines_test)
print("First 10 headlines in training :\n")
pp.pprint(headlines_train_sequence[0:5])
print("First 10 headlines in testing :\n")
pp.pprint(headlines_test_sequence[0:5])

First 10 headlines in training :

[[3989, 3990, 37, 13227, 9250, 5, 7185],
 [5949, 961, 34, 55, 71, 258, 7186, 135, 5, 1146],
 [1147, 4, 62, 186, 16, 13228],
 [3586, 1094, 1696, 116, 2, 388, 275, 199],
 [13, 2, 9251, 842, 397, 477, 27, 2117, 2, 3991]]
First 10 headlines in testing :

[[114, 259, 2, 2599, 30, 8264],
 [46, 3072, 3, 951, 22, 289, 1930, 784, 3435, 1555, 3, 2569, 16, 503, 2221],
 [302, 78, 6237, 167, 1363, 5, 19],
 [38, 74, 85, 4206, 1934, 2544, 2427, 67, 234, 2064, 5, 39, 4, 455, 114, 419],
 [144, 831, 2722, 1637, 1, 17, 297, 5591, 36, 45, 233, 4670]]


# Padding the sequences

In [58]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
headlines_train_sequence_padded , headlines_test_sequence_padded = pad_sequences(headlines_train_sequence , padding="post" , maxlen=max_sentence_length) , pad_sequences(headlines_test_sequence , padding="post" ,  maxlen=max_sentence_length)
print("First 3 headlines padded in training :\n")
pp.pprint(headlines_train_sequence_padded[0:3] )
# print("train shape =" , headlines_train_sequence_padded[0].shape)
print("First 3 headlines padded in testing :\n")
pp.pprint(headlines_test_sequence_padded[0:3])
#print("test shape =" , headlines_test_sequence_padded[0].shape)

max_sentence_length = len(headlines)

First 3 headlines padded in training :

array([[ 3989,  3990,    37, 13227,  9250,     5,  7185,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0],
       [ 5949,   961,    34,    55,    71,   258,  7186,   135,     5,
         1146,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0],
       [ 1147,     4,    62,   186,    16, 13228,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0

# Building The model

In [90]:
print("vocab")
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size , word_embedding_vector_dimension , input_length=max_sentence_length ),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64 , activation="relu"),
    tf.keras.layers.Dense(1 , activation="sigmoid")
])
model.summary()

vocab
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 40, 20)            400000    
                                                                 
 flatten_5 (Flatten)         (None, 800)               0         
                                                                 
 dense_10 (Dense)            (None, 64)                51264     
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 451,329
Trainable params: 451,329
Non-trainable params: 0
_________________________________________________________________


# Compiling and fitting the model

In [100]:

model.compile(optimizer="adam", loss=tf.keras.losses.binary_crossentropy , metrics=["accuracy"] )
model.fit(x=headlines_train_sequence_padded , y=is_sarcastic_train , epochs=5 , validation_data=(headlines_test_sequence_padded , is_sarcastic_test))

Epoch 1/5
668/668 [==============================] - 3s 4ms/step - loss: 4.7426e-04 - accuracy: 0.9999 - val_loss: 1.0028 - val_accuracy: 0.8527
Epoch 2/5
668/668 [==============================] - 3s 4ms/step - loss: 1.3644e-04 - accuracy: 1.0000 - val_loss: 1.0897 - val_accuracy: 0.8471
Epoch 3/5
668/668 [==============================] - 3s 4ms/step - loss: 1.8393e-04 - accuracy: 0.9999 - val_loss: 1.1540 - val_accuracy: 0.8388
Epoch 4/5
668/668 [==============================] - 3s 4ms/step - loss: 7.2157e-06 - accuracy: 1.0000 - val_loss: 1.0888 - val_accuracy: 0.8484
Epoch 5/5
668/668 [==============================] - 4s 6ms/step - loss: 7.5172e-07 - accuracy: 1.0000 - val_loss: 1.1051 - val_accuracy: 0.8474


In [92]:

print(headlines_test[1:2])
model.predict(headlines_test_sequence_padded[1:2])

['report: shame of walking out without buying anything drives 90% of purchases at small businesses']


array([[1.]], dtype=float32)

# Create function that convert a text to padded_sequence to do the prediction

In [99]:
def from_text_to_padded_sequence(tokenizer , texts):
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, padding="post" , maxlen=max_sentence_length)
    return padded_sequences
print("sequence =",from_text_to_padded_sequence(tokenizer ,["Life is boring"]) )
model.predict(from_text_to_padded_sequence(tokenizer ,["Life is boring" ,"kill me please"]))

sequence = [[  64   11 1914    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]]


array([[3.6792755e-03],
       [8.4513260e-07]], dtype=float32)